In [44]:
import pandas as pd
import logging
import os
import sys
from tabulate import tabulate
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from nltk.sentiment import SentimentIntensityAnalyzer

sys.path.append(os.path.abspath(os.path.join('..', 'utils')))  # Adjust the path based on your structure
from logging_configuration import setup_logging
from load_from_csv import load_data
from tabulate_style import tab_fmt
from explode_tokens import explode_to_tokens
from custom_plotly_template import get_custom_layout, set_custom_template

# Call the function to set the custom template
set_custom_template() 

In [48]:
# Initialize Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

def analyze_token_sentiment(tokens_df, sentiment_types=['compound'], return_type='dataframe'):
    """
    Analyzes sentiment for tokens in tokens_df using NLTK's VADER sentiment analysis.

    Parameters:
    - tokens_df: DataFrame containing individual tokens in a 'paragraph' column.
    - sentiment_types: list of str, sentiment scores to calculate, e.g., ['compound', 'pos'].
    - return_type: str, 'dataframe', 'plot', or 'both' for output preference.

    Returns:
    - DataFrame with sentiment scores or None based on return_type.
    """
    # Ensure tokens_df has the correct structure
    if 'paragraph' not in tokens_df.columns:
        logging.error("The DataFrame must contain a 'paragraph' column.")
        return None

    # Filter out non-alphabetic tokens and empty strings
    tokens_df = tokens_df[tokens_df['paragraph'].str.isalpha() & tokens_df['paragraph'].str.strip().astype(bool)].copy()

    # Calculate the length (character count) of each token
    tokens_df['length'] = tokens_df['paragraph'].str.len()

    # Initialize sentiment score columns
    for sentiment_type in sentiment_types:
        tokens_df[sentiment_type] = None  # Initialize columns

    # Apply sentiment analysis to each token with error handling
    for i, token in enumerate(tokens_df['paragraph']):
        try:
            scores = sia.polarity_scores(token)
            for sentiment_type in sentiment_types:
                tokens_df.at[i, sentiment_type] = scores[sentiment_type]
        except Exception as e:
            logging.error(f"Error calculating sentiment for token index {i}: {e}")
            for sentiment_type in sentiment_types:
                tokens_df.at[i, sentiment_type] = None

    # Calculate the 95th percentile for each sentiment type and log the values
    for sentiment_type in sentiment_types:
        if sentiment_type in tokens_df.columns:
            percentile_95_sentiment = round(tokens_df[sentiment_type].quantile(0.95), 2)
            logging.info(f"95th Percentile Sentiment Score ({sentiment_type}): {percentile_95_sentiment}")

            # Filter tokens with sentiment >= 95th percentile
            high_sentiment_tokens = tokens_df[tokens_df[sentiment_type] >= percentile_95_sentiment]
            logging.info("Top tokens by sentiment score:\n")
            logging.info(high_sentiment_tokens.head())

            # Plot sentiment distribution if required
            if return_type in ['plot', 'both']:
                fig = px.histogram(tokens_df, x=sentiment_type, nbins=30,
                                   title=f'Distribution of {sentiment_type.capitalize()} Sentiment Scores',
                                   labels={sentiment_type: 'Sentiment Score', 'count': 'Count'},
                                   color_discrete_sequence=px.colors.sequential.Jet)  # Using 'Jet' color scale
                fig.show()

    return tokens_df if return_type in ['dataframe', 'both'] else None

In [46]:
df = load_data()

2024-10-27 18:23:47,522 - INFO - cagliostro_gutenberg.csv imported successfully!
2024-10-27 18:23:47,525 - INFO - There are 1775 rows and 8 columns.


2024-10-27 18:23:47,556 - INFO - Schema of the loaded dataset:
2024-10-27 18:23:47,565 - INFO - 
+----+---------------+-------------+------------+
|    | Column Name   | Data Type   |   n_unique |
|----+---------------+-------------+------------|
|  0 | id            | int64       |       1775 |
|  1 | chapter_title | object      |          1 |
|  2 | paragraph     | object      |        908 |
|  3 | quote         | float64     |          0 |
|  4 | source_url    | object      |          1 |
|  5 | created_at    | float64     |          0 |
|  6 | title         | object      |          1 |
|  7 | content       | float64     |          0 |
+----+---------------+-------------+------------+
2024-10-27 18:23:47,573 - WARNING - Column 'quote' has no unique values and will be dropped.
2024-10-27 18:23:47,581 - WARNING - Column 'created_at' has no unique values and will be dropped.
2024-10-27 18:23:47,587 - WARNING - Column 'content' has no unique values and will be dropped.


In [50]:
tokens_df = explode_to_tokens(df, column='paragraph')
tokens_df = pd.DataFrame(tokens_df)
#tab_fmt(tokens_df, 5, style='psql')

In [ ]:
# Call the analyze_token_sentiment function
result = analyze_token_sentiment(tokens_df, sentiment_types=['compound', 'pos'], return_type='both')